# Idenfication of nearest ORFs

In [1]:
import os
import sys
import pandas as pd
print(sys.version)

3.11.3 (main, May 15 2023, 10:43:03) [Clang 14.0.6 ]


# Identify additional ORFs adjacent to the nearest ORF for each peak
### The search is limited to 1000 bp each direction of the peak 

In [2]:
py_script = "~/GitProjects/geisingerlab/geisingerlab-chipseq-bfmrs/src/nearest_ORF_additional.py"
infile = "~/2023_ChipSeq/average_peak_summit/BfmR-ChIP-49_seed1.average_summit.bed"
outfile = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.tsv"
annotations = "~/2023_ChipSeq/annotations"

In [5]:
cmd = ' '.join(['python', py_script, infile, outfile, annotations])
os.system(cmd)

0

In [130]:
!wc -l $outfile

     332 /Users/yunfei/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.RNASeq.activate_or_repress.tsv


# Filter matched ORFs by activated / repressed call from RNAseq table

In [128]:
rnaseq_bfmRS = "~/2023_ChipSeq/rnaseq/bfmRS_WT.xlsx"
rnaseq_bfmR = "~/2023_ChipSeq/rnaseq/bfmR_WT.xlsx"
rnaseq_bfmS = "~/2023_ChipSeq/rnaseq/bfmS_WT.xlsx"

ORF_table = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.tsv"
ORF_RNAseq = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.RNASeq.tsv"


In [129]:
def read_rnaseq_table(file):
    df = pd.read_excel(file)
    df = df.set_index("gene_id")
    return df

In [99]:
rnaseq_table = {}
rnaseq_table.update({"BfmRS_WT" : read_rnaseq_table(rnaseq_bfmRS)})
rnaseq_table.update({"BfmR_WT" : read_rnaseq_table(rnaseq_bfmR)})
rnaseq_table.update({"BfmS_WT" : read_rnaseq_table(rnaseq_bfmS)})

df_chipseq = pd.read_csv(ORF_table, sep='\t') # remove the index option to keep summit_pos column
df_chipseq = df_chipseq.set_index("locus_tag")

col_names = ["log2(fold_change)", "significant", "protein_id", "product"]

for i,j in rnaseq_table.items():
    df_rnaseq = j
    for row, column in df_chipseq.iterrows():
        for k in col_names:
            new_column_name = i + "_" + k
            try:
                df_chipseq.loc[row, new_column_name] = df_rnaseq.loc[row,k]
            except:
                pass
df_chipseq.to_csv(ORF_RNAseq, sep='\t')
df_chipseq

/Users/yunfei/opt/anaconda3/envs/ChipSeq/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/yunfei/opt/anaconda3/envs/ChipSeq/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
/Users/yunfei/opt/anaconda3/envs/ChipSeq/lib/python3.11/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,summit_pos,Nth nearest ORF,chrom,start,end,strand,distance_to_match,intergenic_distance,match_type,average_fold_enrichment,...,BfmRS_WT_protein_id,BfmRS_WT_product,BfmR_WT_log2(fold_change),BfmR_WT_significant,BfmR_WT_protein_id,BfmR_WT_product,BfmS_WT_log2(fold_change),BfmS_WT_significant,BfmS_WT_protein_id,BfmS_WT_product
locus_tag,,,,,,,,,,,,,,,,,,,,,
ACX60_RS00020,3649,0,NZ_CP012004.1,3882,6351,1,-233,0,coding,2.95,...,WP_000093728.1,DNA gyrase subunit B,-0.622859,yes,WP_000093728.1,DNA gyrase subunit B,-0.128250,no,WP_000093728.1,DNA gyrase subunit B
A1S_3475,7132,0,NC_009084.1,6196,8608,1,936,0,coding,1.49,...,YP_001083098.1,hypothetical protein,1.417840,yes,YP_001083098.1,hypothetical protein,0.426927,no,YP_001083098.1,hypothetical protein
ACX60_RS18245,11064,0,NZ_CP012005.1,10999,10429,-1,-65,0,intergenic,4.16,...,WP_000287492.1,hypothetical protein,1.262890,yes,WP_000287492.1,hypothetical protein,-0.281466,no,WP_000287492.1,hypothetical protein
ACX60_RS18240,11064,1,NZ_CP012005.1,10427,9890,-1,-637,572,intergenic,4.16,...,WP_000123889.1,hypothetical protein,0.984207,yes,WP_000123889.1,hypothetical protein,-1.298720,yes,WP_000123889.1,hypothetical protein
A1S_3469,11305,0,NC_009083.1,10723,10366,-1,-582,0,intergenic,2.02,...,YP_001083093.1,diaminopimelate decarboxylase,1.398850,yes,YP_001083093.1,diaminopimelate decarboxylase,0.297624,no,YP_001083093.1,diaminopimelate decarboxylase
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACX60_RS18155,3851671,0,NZ_CP012004.1,3851613,3850914,-1,-58,0,intergenic,2.46,...,WP_000592160.1,thiol:disulfide interchange protein,-1.093730,yes,WP_000592160.1,thiol:disulfide interchange protein,1.383670,yes,WP_000592160.1,thiol:disulfide interchange protein
ACX60_RS18190,3857195,0,NZ_CP012004.1,3857162,3857027,-1,-33,0,intergenic,2.09,...,WP_000831329.1,50S ribosomal protein L34,-1.169890,yes,WP_000831329.1,50S ribosomal protein L34,0.352361,no,WP_000831329.1,50S ribosomal protein L34
ACX60_RS18185,3857195,1,NZ_CP012004.1,3856998,3856605,-1,-197,164,intergenic,2.09,...,WP_001240371.1,ribonuclease P protein component,-1.381460,yes,WP_001240371.1,ribonuclease P protein component,0.207498,no,WP_001240371.1,ribonuclease P protein component


# Filter out matched ORFs with the same direction of regulation
## Generate Activated or repressed calls

In [100]:
py_activated_repressed = "~/GitProjects/geisingerlab/geisingerlab-chipseq-bfmrs/src/activate_or_repressed.py"
condition_table = "~/2023_ChipSeq/repressed_or_activated.txt"
infile = ORF_RNAseq
outfile = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.RNASeq.activate_or_repress.tsv"

cmd = " ".join(["python", py_activated_repressed, infile, condition_table, outfile])
os.system(cmd)

0

In [126]:
df_mastertable = pd.read_csv(outfile, sep='\t')
# build a dictionary of {locus tag : nearest ORF activated or repressed}
direction_dict = {}
df_nearest_ORF = df_mastertable[df_mastertable['Nth nearest ORF'] == 0]
df_nearest_ORF = df_mastertable[['summit_pos', 'general call']]
for summit in df_nearest_ORF['summit_pos'].tolist():
    general_call = df_nearest_ORF[df_nearest_ORF['summit_pos']==summit]['general call'].iloc[0]
    direction_dict.update({summit:general_call})

filtered_summit_list = []
for summit, direction in direction_dict.items(): # iterate by summit_pos
    df_summit = df_mastertable[df_mastertable['summit_pos'] == summit]
    df_summit = df_summit[['summit_pos', 'locus_tag', 'Nth nearest ORF', 'general call']]
    if df_summit.shape[0] > 1: # if more than one match
        directions = df_summit['general call'].tolist()
        if len(set(directions)) > 1: # if directions are different, add to the list of values to drop
            filtered_summit_list.append(summit)
condition = (df_mastertable['summit_pos'].isin(filtered_summit_list)) & (df_mastertable['Nth nearest ORF'] != 0)
df_mastertable = df_mastertable[~condition]
df_mastertable = df_mastertable.set_index('summit_pos')
filtered_csv = "~/2023_ChipSeq/peak_stat_next_ORFs/BfmR-ChIP-49_seed1.nextORFs.RNASeq.activate_or_repress.filtered.tsv"
df_mastertable.to_csv(filtered_csv, sep='\t', index=True)

In [127]:
df_mastertable

,locus_tag,Nth nearest ORF,chrom,start,end,strand,distance_to_match,intergenic_distance,match_type,average_fold_enrichment,...,BfmR_WT_significant,BfmR_WT_protein_id,BfmR_WT_product,BfmS_WT_log2(fold_change),BfmS_WT_significant,BfmS_WT_protein_id,BfmS_WT_product,condition_code,general call,specific call
summit_pos,,,,,,,,,,,,,,,,,,,,,
3649,ACX60_RS00020,0,NZ_CP012004.1,3882,6351,1,-233,0,coding,2.95,...,yes,WP_000093728.1,DNA gyrase subunit B,-0.128250,no,WP_000093728.1,DNA gyrase subunit B,"(0, -1, 0)",activated,class B
7132,A1S_3475,0,NC_009084.1,6196,8608,1,936,0,coding,1.49,...,yes,YP_001083098.1,hypothetical protein,0.426927,no,YP_001083098.1,hypothetical protein,"(1, 1, 0)",repressed,class B
11064,ACX60_RS18245,0,NZ_CP012005.1,10999,10429,-1,-65,0,intergenic,4.16,...,yes,WP_000287492.1,hypothetical protein,-0.281466,no,WP_000287492.1,hypothetical protein,"(1, 1, 0)",repressed,class B
11064,ACX60_RS18240,1,NZ_CP012005.1,10427,9890,-1,-637,572,intergenic,4.16,...,yes,WP_000123889.1,hypothetical protein,-1.298720,yes,WP_000123889.1,hypothetical protein,"(1, 1, -1)",repressed,class A
11305,A1S_3469,0,NC_009083.1,10723,10366,-1,-582,0,intergenic,2.02,...,yes,YP_001083093.1,diaminopimelate decarboxylase,0.297624,no,YP_001083093.1,diaminopimelate decarboxylase,"(1, 1, 0)",repressed,class B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3851671,ACX60_RS18155,0,NZ_CP012004.1,3851613,3850914,-1,-58,0,intergenic,2.46,...,yes,WP_000592160.1,thiol:disulfide interchange protein,1.383670,yes,WP_000592160.1,thiol:disulfide interchange protein,"(-1, -1, 1)",activated,class A
3857195,ACX60_RS18190,0,NZ_CP012004.1,3857162,3857027,-1,-33,0,intergenic,2.09,...,yes,WP_000831329.1,50S ribosomal protein L34,0.352361,no,WP_000831329.1,50S ribosomal protein L34,"(0, -1, 0)",activated,class B
3857195,ACX60_RS18185,1,NZ_CP012004.1,3856998,3856605,-1,-197,164,intergenic,2.09,...,yes,WP_001240371.1,ribonuclease P protein component,0.207498,no,WP_001240371.1,ribonuclease P protein component,"(-1, -1, 0)",activated,class B
